# S-3: 성적 합계 및 평균

In [1]:
import pyspark

myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
        .builder\
        .master("local")\
        .config(conf=myConf)\
        .appName("s-3")\
        .getOrCreate()

21/10/02 14:15:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Original Data

In [2]:
testList = [
    ('김하나', 'English', 100),
    ('김하나', 'Math', 80),
    ('임하나', 'English', 70),
    ('임하나', 'Math', 100),
    ('김갑돌', 'English', 82.3),
    ('김갑돌', 'Math', 98.5)
           ]
testRdd = spark.sparkContext.parallelize(testList)
testRdd.collect()

[('김하나', 'English', 100),
 ('김하나', 'Math', 80),
 ('임하나', 'English', 70),
 ('임하나', 'Math', 100),
 ('김갑돌', 'English', 82.3),
 ('김갑돌', 'Math', 98.5)]

### 문제 3-1: 이름으로 합계 구하기

In [3]:
testRddMapped = testRdd.map(lambda x: (x[0], x[2]))
testRddCbk = testRddMapped.combineByKey(lambda value: (value),
                     lambda x, value: (x + value),
                     lambda x, y: (x + y)
                    )
for elem in testRddCbk.collect():
    print(f"'{elem[0]}' {elem[1]}")

/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


'김하나' 180
'임하나' 170
'김갑돌' 180.8


### 문제 3-2: 과목으로 합계 구하기(과목을 key로 사용한다.)

In [4]:
testRddMapped = testRdd.map(lambda x: (x[1], x[2]))
testRddCbk = testRddMapped.combineByKey(lambda value: (value),
                                        lambda x, value: (x+value),
                                        lambda x, y: (x+y)
                                       )
for elem in testRddCbk.collect():
    print(f"'{elem[0]}' {elem[1]}")

'English' 252.3
'Math' 278.5


### 문제 3-3: 이름으로 합계와 개수를 구한다. (이름을 key로한다.)

In [15]:
testRddMapped = testRdd.map(lambda x: (x[0], x[2])) # (name, score)
testRddCbk = testRddMapped.combineByKey(
            lambda value: (value, 1), # name: (score, count)
            lambda x, value: (x[0] + value, x[1] + 1), # (_score + score, value + 1)
            lambda x, y: (x[0]+y[0], x[1]+y[1])
)

for elem in testRddCbk.collect():
    print(f"'{elem[0]}' {elem[1]}")

'김하나' (180, 2)
'임하나' (170, 2)
'김갑돌' (180.8, 2)


/opt/homebrew/Cellar/apache-spark/3.1.2/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


### 문제 3-4: 이름으로 평균 계산하기 (이름을 key로 한다.)

In [17]:
testRddResult = testRddCbk.map(
    lambda x: (x[0], x[1][0] / x[1][1])
)

for elem in testRddResult.collect():
    print(f"'{elem[0]}' {elem[1]}")

'김하나' 90.0
'임하나' 85.0
'김갑돌' 90.4


In [19]:
spark.stop()
print('Done')

Donw
